<a href="https://colab.research.google.com/github/aditya02shah/NLP/blob/main/Sentiment_Analysis_Using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.0 MB/s eta 0:00:00


##Importing Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import re

##Importing BERT Transformer

In [3]:
import tensorflow as tf
from transformers import AutoTokenizer, TFBertForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [4]:
text = 'It is the best.'
encoded_text = tokenizer.encode(text, return_tensors='tf')

result = model(encoded_text)

predicted_class = tf.argmax(result.logits, axis=1).numpy()[0]

sentiment_label = int(predicted_class) + 1

print("Predicted Sentiment Label:", sentiment_label)

Predicted Sentiment Label: 5


##Web Scraping Reviews from Yelp

In [5]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [6]:
reviews

['Some of the best Milkshakes me and my daughter ever tasted. MMMMMM HMMMMMMMM.',
 "Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. We all enjoyed our food, as well as our coffee and tea drinks.We were greeted immediately by a friendly server asking if we would like to sit inside or out. We said we would like inside, but weren't exactly sure how many were joining us yet- at least 4. We were told this was no problem, the more the merrier. A few minutes later when 4 more joined our party and we explained to the server we had 6, he just quickly switched our table. I really enjoyed my serenity tea, just what I needed after a long flight in from Sfo that morning. Everyone else were more interested in the lattes for expresso drinks. All said they were hot and delicious. 2 of us ordered the avo on toast. So yummy with the beetroot... I will start adding this to mine now at home, and have f

In [7]:
import numpy as np
import pandas as pd

In [8]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [9]:
df['review'].iloc[2]

'Great place with delicious food and friendly staff. It is small but has outdoor seating and a relaxed ambiance. Perfect place to enjoy a cup of coffee. I am visiting Sydney for the first time but this place seems like is a local favorite.'

##Analysing Sentiment from Reviews

In [10]:
def sentiment_score(review):
    encoded_text = tokenizer.encode(review, return_tensors='tf')
    result = model(encoded_text)
    predicted_class = tf.argmax(result.logits, axis=1).numpy()[0]
    sentiment_label = int(predicted_class)+1
    return sentiment_label

In [11]:
sentiment_score("It is the worst thing.Servive is really bad")

1

In [12]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [13]:
df

,review,sentiment
0,Some of the best Milkshakes me and my daughter...,5
1,Six of us met here for breakfast before our wa...,4
2,Great place with delicious food and friendly s...,5
3,Great food amazing coffee and tea. Short walk ...,5
4,It was ok. Had coffee with my friends. I'm new...,3
5,Ricotta hot cakes! These were so yummy. I ate ...,5
6,Great staff and food. Must try is the pan fri...,5
7,We came for brunch twice in our week-long visi...,4
8,I came to Social brew cafe for brunch while ex...,5
9,It was ok. The coffee wasn't the best but it w...,3


In [14]:
df['review'].iloc[3]

'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly'